# Работа с базами данных

## Лабораторная работа 9

In [1]:
import sqlite3
import pandas as pd

1. Создайте файл базы данных c названием `recipes.db`. Создайте объект-курсор. 

In [22]:
con = sqlite3.connect('recipes.db') # Открываем базу данных 
cur = con.cursor() # Создаем объект-курсор

2. Напишите и выполните SQL-запрос для создания таблицы рецептов `Recipe`. Таблица должна содержать следующие поля:
`id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`. Определитесь с типами и составом ключевых полей.

In [3]:
table1  = """
CREATE TABLE if not exists Recipe 
(
    ID INTEGER PRIMARY KEY NOT NULL, 
    NAME VARCHAR(200)  NOT NULL,
    MINUTES INTEGER NOT NULL,
    SUBMITTED INTEGER NOT NULL,
    DESCRIPTION TEXT,
    N_INGREDIENTS INTEGER NOT NULL   
); """
try:  # Обрабатываем исключения 
    cur.executescript(table1)  # Выnолняем SQL-зanpocы 
except sqlite3.DatabaseError as  err: 
    print("Ошибка:", err) 
else: 
    print ("Запрос успешно выполнен") 

Запрос успешно выполнен


3. Напишите и выполните SQL-запрос для создания таблицы отзывов `Review`. Таблица должна содержать следующие поля:
`id`, `user_id`, `recipe_id`, `date`, `rating`, `review`. Определитесь с типами полей, набором ключевых полей. При помощи внешнего ключа соедините две таблицы.

In [4]:
table2  = """
CREATE TABLE if not exists Review
(
    ID INTEGER PRIMARY KEY NOT NULL,
    USER_ID INTEGER NOT NULL,
    RECIPE_ID INTEGER NOT NULL,
    DATE DATE,
    RATING INTEGER,
    REVIEW TEXT,
    FOREIGN KEY(RECIPE_ID) REFERENCES Recipe(ID)  
); """
try:  # Обрабатываем исключения 
    cur.executescript(table2)  # Выnолняем SQL-зanpocы 
except sqlite3.DatabaseError as  err: 
    print("Ошибка:", err) 
else: 
    print ("Запрос успешно выполнен") 

Запрос успешно выполнен


4. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample_with_tags_ingredients.csv` (__ЛР5__) в созданные таблицы

In [5]:
recipe = pd.read_csv('recipes_sample_with_tags_ingredients.csv', sep=',')

for i, row in recipe.iterrows():
    data = row.loc[['id','name', 'minutes', 'submitted', 'description', 'n_ingredients']]
    cur.execute(
        '''
        INSERT INTO
        recipe (id, name, minutes, submitted, description, n_ingredients)
        VALUES(?, ?, ?, ?, ?, ?)
    ''',data
    )
con.commit()

In [9]:
review = pd.read_csv('reviews_sample.csv', sep=',')

for i, row in review.iterrows():
    data = row.loc[['user_id', 'recipe_id', 'date', 'rating', 'review']]
    cur.execute(
        '''
        INSERT INTO
        review (user_id, recipe_id, date, rating, review)
        VALUES(?, ?, ?, ?, ?)
    ''',data
    )
con.commit()

5. Найдите все рецепты, для выполнения которых нужно ровно 10 ингредиентов. Выведите на экран первые 5 из найденных рецептов.

In [10]:
query_1 = '''
select *
from recipe
where n_ingredients = 10;
'''
cur.execute(query_1) # выполняем запрос
fa = cur.fetchall() # возвращает список всех записей из результата запроса
colnames = [i[0] for i in cur.description] #cursor.description предоставляет имена столбцов последнего запроса
result = pd.DataFrame(fa,columns= colnames)

result.head()

,ID,NAME,MINUTES,SUBMITTED,DESCRIPTION,N_INGREDIENTS
0,176,cola cake,55,1999-08-22,None,10
1,246,lee s hot crab dip,45,1999-09-01,lee's hot crab dip,10
2,289,feijoada brazilian bean soup ii,150,1999-10-04,brazilian bean soup,10
3,373,tiramisu balducci s in new york city,95,1999-08-17,this is an adopted recipe. i shall be trying i...,10
4,544,greek mushroom salad,38,1999-08-10,None,10


6. Найдите название рецепта, для выполнения которого требуется больше всего времени.

In [11]:
query_2 = '''
SELECT name
FROM recipe
WHERE minutes = (
    SELECT MAX(minutes)
    FROM recipe
)
'''
cur.execute(query_2) # выполняем запрос
fa = cur.fetchall()  # возвращает список всех записей из результата запроса
colnames = [i[0] for i in cur.description] #cursor.description предоставляет имена столбцов последнего запроса
result = pd.DataFrame(fa,columns= colnames)

result.head()

,NAME
0,strawberry liqueur
1,blueberry liqueur


7. Запросите у пользователя id рецепта и верните информацию об этом рецепте. Если рецепт отсуствует, выведите соответствующее сообщение.

In [13]:
recipe_id = input("Введите id рецепта: ")

query_3 = '''
SELECT name
FROM recipe
WHERE id = ?
'''

cur.execute(query_3, (recipe_id,))
fa = cur.fetchall()
colnames = [i[0] for i in cur.description] 
result = pd.DataFrame(fa,columns= colnames)

if result.empty:
    print('Рецепт отсутствует')
else:
    print(result)

Введите id рецепта: 292568
               NAME
0  better  cake mix


8. Найдите кол-во отзывов с рейтингом 5.

In [14]:
query_4 = '''
SELECT COUNT(id)
FROM review
WHERE rating = 5
'''
cur.execute(query_4)
fa = cur.fetchall()
colnames = [i[0] for i in cur.description] 
result = pd.DataFrame(fa,columns= colnames)

result.head()

,COUNT(id)
0,91361


9. Найдите кол-во уникальных рецептов, не имеющих отзывов с рейтингом, меньше 4. 

In [27]:
query_4 = '''
SELECT DISTINCT COUNT(recipe.id) FROM recipe 
LEFT JOIN review
ON recipe.id = review.recipe_id
WHERE review.rating <= 4
'''
cur.execute(query_4)
fa = cur.fetchall()
colnames = [i[0] for i in cur.description] #cursor.description предоставляет имена столбцов последнего запроса
result = pd.DataFrame(fa,columns=colnames)
result.head()

,COUNT(recipe.id)
0,35335


10. Найдите кол-во рецептов, опубликованных в 2010 году и имеющих длину не менее 15 минут.

In [24]:
query_5 = '''
SELECT count(*) FROM RECIPE
WHERE minutes >= 15 AND
submitted BETWEEN '2010-01-01' AND '2010-12-31'
'''
cur.execute(query_5) # выполняем запрос 
fa = cur.fetchall() # возвращает список всех записей из результата запроса
colnames = [i[0] for i in cur.description] 
result = pd.DataFrame(fa,columns= colnames)

result.head()

,count(*)
0,1319


11. Выберите id рецепта, название рецепта, id пользователя, оставившего отзыв, дату отзыва и рейтинг для тех рецептов, которые имеют не менее 3 ингредиентов. Отсортируйте результат по id рецепта.

In [19]:
query_6 = '''
SELECT rec.id, rec.name, rev.user_id, rev.date, rev.rating
FROM recipe rec
LEFT JOIN review rev
ON rec.id = rev.recipe_id
WHERE rec.n_ingredients >= 3
ORDER BY rec.id
'''
cur.execute(query_6)
fa = cur.fetchall()
colnames = [i[0] for i in cur.description] 
result = pd.DataFrame(fa,columns= colnames)

result.head()

,ID,NAME,USER_ID,DATE,RATING
0,48,boston cream pie,32421.0,2002-03-15,0.0
1,48,boston cream pie,68674.0,2004-05-03,2.0
2,55,betty crocker s southwestern guacamole dip,53959.0,2006-01-12,4.0
3,55,betty crocker s southwestern guacamole dip,165567.0,2006-03-31,5.0
4,55,betty crocker s southwestern guacamole dip,851190.0,2010-05-23,5.0


In [20]:
cur.close() # Закрываем объект-курсор
con.close() # Закрываем соединение